In [1]:
import gmaps
from config import g_key
import pandas as pd 
import requests

In [2]:
challenge_gmaps_df = pd.read_csv('WeatherPy_challenge.csv')
challenge_gmaps_df.dtypes

City_ID                       int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [3]:
# user prompts for input min and max temp range and whether rain/snow or not
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    
raining_prompt = input("Do you want it to be raining? (yes/no) ")
snowing_prompt = input("Do you want it to be snowing? (yes/no) ")

# Filter DataFrames based on input and nested decision statements, and logical expressions
if (raining_prompt == 'yes') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'yes') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
else:
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
    
preferred_challenges_df.head()

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,amapa,HN,2020-01-09 01:29:32,15.09,-87.97,71.60,94,40,4.70,scattered clouds,0.0,0.0
9,9,busselton,AU,2020-01-09 01:29:02,-33.64,115.35,71.01,51,17,1.01,few clouds,0.0,0.0
17,17,bambous virieux,MU,2020-01-09 01:29:38,-20.34,57.76,75.20,88,40,12.50,scattered clouds,0.0,0.0
18,18,georgetown,GY,2020-01-09 01:28:43,6.80,-58.16,78.80,88,40,9.17,scattered clouds,0.0,0.0
22,22,tutoia,BR,2020-01-09 01:26:56,-2.76,-42.27,75.65,92,10,3.62,clear sky,0.0,0.0


In [4]:
preferred_challenges_df.count()

City_ID                     110
City                        110
Country                     110
Date                        110
Lat                         110
Lng                         110
Max Temp                    110
Humidity                    110
Cloudiness                  110
Wind Speed                  110
Current Description         110
Rain inches (last 3 hrs)    110
Snow inches (last 3 hrs)    110
dtype: int64

In [6]:
# clean up the DataFrame by dropna() funtion
clean_preferred_challenges_df = preferred_challenges_df.dropna()
clean_preferred_challenges_df.count()
# make a copy for a new DataFrame 
challenge_hotel_df = clean_preferred_challenges_df[["City", "Country","Max Temp",  "Current Description","Lat", "Lng"]].copy()
challenge_hotel_df['Hotel Name'] = ''

# search nearest hotel name by using google place API 
for index, row in challenge_hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params = {'key': g_key, 'type':'lodging', 'radius' : 5000}
    params['location'] = f"{lat},{lng}"
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        challenge_hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

# Save and upload the new DataFrame as csv file
challenge_hotel_df.to_csv(path_or_buf='weatherPy_vacation.csv', index_label='City_ID')

challenge_hotel_df.head(10)

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,amapa,HN,71.60,scattered clouds,15.09,-87.97,Incubadora GAHSA
9,busselton,AU,71.01,few clouds,-33.64,115.35,Observatory Guest House
17,bambous virieux,MU,75.20,scattered clouds,-20.34,57.76,Casa Tia Villa
18,georgetown,GY,78.80,scattered clouds,6.80,-58.16,Kanhai's Center
22,tutoia,BR,75.65,clear sky,-2.76,-42.27,Pousada Guará's
31,bluff,AU,84.06,few clouds,-23.58,149.07,Bluff Hotel
44,fonte boa,BR,79.50,broken clouds,-2.51,-66.09,Hotel Dona Antonia
65,kapaa,US,84.20,broken clouds,22.08,-159.32,Sheraton Kauai Coconut Beach Resort
72,atuona,PF,82.42,few clouds,-9.80,-139.03,Villa Enata
74,port alfred,ZA,70.00,overcast clouds,-33.59,26.89,The Halyards Hotel


In [7]:
# create a  marker layer map with a pop-up info box by gmaps dependence
gmaps.configure(api_key = g_key)
locations = challenge_hotel_df[['Lat','Lng']]
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>'''

hotel_info = [info_box_template.format(**row) for index, row in challenge_hotel_df.iterrows() ]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))